In [11]:
import numpy as np
import pandas as pd


In [12]:
# Load Excel file (Install openpyxl for loading (!pip install openpyxl))
df = pd.read_excel("D:\\IndiaAI\\Champions Code\\Dataset\\Champion_Data.xlsx")
#df[['Gims_CategoryName','GrievanceDescription', 'FinalReply']].head(15)

df.head(15)

,GrievanceNo,Gims_CategoryName,State,CreatedDate,ReplyStatus,GrievanceDescription,FinalReply,FinalReplyDate
0,UP00135828,UAM/Udyam Registration/Certificate related issues,UTTAR PRADESH,2024-12-30,Replied,Please find the attachment of Letter Head,"Sir,&#13;&#10; Please upload the clear im...",2024-12-31
1,BR00135827,UAM/Udyam Registration/Certificate related issues,BIHAR,2024-12-30,Replied,Udyam no Udyam-br-31-0038425 My address i...,"Sir,&#13;&#10;As per your request you are advi...",2024-12-31
2,MH00135826,UAM/Udyam Registration/Certificate related issues,MAHARASHTRA,2024-12-30,Replied,"To, The Udyog Aadhaar Registration Authority,...",&#2350;&#2361;&#2379;&#2342;&#2351;/&#2350;&#2...,2025-01-01
3,CG00135825,UAM/Udyam Registration/Certificate related issues,CHHATTISGARH,2024-12-30,Replied,"This is regarding Udyam Registration, even aft...",With the provided PAN the registered Udyam is ...,2025-01-03
4,CG00135824,UAM/Udyam Registration/Certificate related issues,CHHATTISGARH,2024-12-30,Replied,The grievance is regarding my Udyam Registrati...,With the provided PAN the registered Udyam is ...,2025-01-03
5,JH00135823,UAM/Udyam Registration/Certificate related issues,JHARKHAND,2024-12-30,Replied,"DEAR SIR, I AM A PROPRIETOR HAVING PAN: GZDPD7...","Madam,&#13;&#10;This is regarding your grievan...",2025-01-02
6,OD00135822,UAM/Udyam Registration/Certificate related issues,ODISHA,2024-12-30,Replied,When I apply for udyam registration.. It shows...,"Sir, the following information is available in...",2024-12-31
7,UP00135821,UAM/Udyam Registration/Certificate related issues,UTTAR PRADESH,2024-12-30,Replied,सर / मैडम मैं सुभम अग्रवाल पुत्र फते चन्द्र ...,"Sir,&#13;&#10; As per our records an UAM is re...",2024-12-31
8,RJ00135820,UAM/Udyam Registration/Certificate related issues,RAJASTHAN,2024-12-30,Replied,I am running a Cement Business and want to get...,Your Udyam registration Number is UDYAM-RJ-17...,2025-01-01
9,RJ00135819,Related to DCMSME Scheme,RAJASTHAN,2024-12-30,Replied,Sir mera pm vishwakarma yojna ka OTP nhi aa rh...,The Tool kit will be distributed through India...,2025-01-01


In [13]:
# Print categories in Gims_CategoryName column and their counts
df['Gims_CategoryName'].value_counts()

Gims_CategoryName
UAM/Udyam Registration/Certificate related issues                    7663
Delayed Payment/MSEFC related issues                                  524
Loans from Banks                                                      194
Related to Scheme of KVIC                                             105
Related to MSME Scheme                                                 89
Related to  MSME-DFO                                                   72
MSME Act 2006 related Matters                                          48
Any other banking issue                                                42
Related to Public Procurement by PSUs                                  28
Related to DCMSME Scheme                                               28
Related to GST                                                         25
Related to various State Government                                    24
Insurance Claim related issues                                         16
Definition of MSMEs 

In [14]:
!pip -q install deep-translator langdetect

In [15]:
# Translating GrievanceDescription & FinalReply to English (Hindi/Hinglish/other → English) including lowercasing
# Uses existing libraries: deep-translator (GoogleTranslator) + langdetect for language detection.
# Note: Translation requires internet access and may be rate-limited by the provider.
# Install langauge detection and translation libraries if not already installed:(!pip install deep-translator langdetect)
import re
from functools import lru_cache
from deep_translator import GoogleTranslator
from langdetect import detect, LangDetectException

CATEGORY_COL = "Gims_CategoryName"
DESC_COL = "GrievanceDescription"
REPLY_COL = "FinalReply"

# Quick script detection for Hindi (Devanagari block)
DEVANAGARI_RE = re.compile(r"[\u0900-\u097F]")

def _safe_str(x) -> str:
    if x is None or (isinstance(x, float) and pd.isna(x)) or pd.isna(x):
        return ""
    return str(x)

@lru_cache(maxsize=50000)
def _translate_cached(text: str) -> str:
    # GoogleTranslator auto-detects source when source='auto'
    return GoogleTranslator(source="auto", target="en").translate(text)

def to_english(text: str) -> str:
    text = _safe_str(text).strip()
    if not text:
        return ""
    # If the text contains Hindi characters, translate.
    if DEVANAGARI_RE.search(text):
        try:
            return _translate_cached(text)
        except Exception:
            return text
    # Otherwise, detect language; translate if not English.
    try:
        lang = detect(text)
    except LangDetectException:
        lang = "unknown"
    if lang and lang != "en" and lang != "unknown":
        try:
            return _translate_cached(text)
        except Exception:
            return text
    # Hinglish may be detected as 'en' sometimes; you can force-translate later if needed.
    return text

# Defensive checks
missing_cols = [c for c in [DESC_COL, REPLY_COL] if c not in df.columns]
if missing_cols:
    raise KeyError(f"Missing expected columns in df: {missing_cols}. Available columns: {list(df.columns)}")

# Translate into English columns
df[CATEGORY_COL] = df[CATEGORY_COL].apply(to_english)
df[DESC_COL] = df[DESC_COL].apply(to_english)
df[REPLY_COL] = df[REPLY_COL].apply(to_english)

# Optional: standardize to lowercase after translation (English output)
df[CATEGORY_COL] = df[CATEGORY_COL].str.lower()
df[DESC_COL] = df[DESC_COL].str.lower()
df[REPLY_COL] = df[REPLY_COL].str.lower()
df[[CATEGORY_COL,DESC_COL, REPLY_COL]].head(10)

,Gims_CategoryName,GrievanceDescription,FinalReply
0,uam/udyam registration/certificate related issues,please find the attachment of letter head,"sir,&#13;&#10; please upload the clear im..."
1,uam/udyam registration/certificate related issues,udyam no udyam-br-31-0038425 my address i...,"sir,&#13;&#10;as per your request you are advi..."
2,uam/udyam registration/certificate related issues,"to, the udyog aadhaar registration authority,...",&#2350;&#2361;&#2379;&#2342;&#2351;/&#2350;&#2...
3,uam/udyam registration/certificate related issues,"this is regarding udyam registration, even aft...",with the provided pan the registered udyam is ...
4,uam/udyam registration/certificate related issues,the grievance is regarding my udyam registrati...,with the provided pan the registered udyam is ...
5,uam/udyam registration/certificate related issues,"dear sir, i am a proprietor having pan: gzdpd7...","madam,&#13;&#10;this is regarding your grievan..."
6,uam/udyam registration/certificate related issues,when i apply for udyam registration.. it shows...,"sir, the following information is available in..."
7,uam/udyam registration/certificate related issues,sir/madam i am subham aggarwal son of fateh ch...,"sir,&#13;&#10; as per our records an uam is re..."
8,uam/udyam registration/certificate related issues,i am running a cement business and want to get...,your udyam registration number is udyam-rj-17...
9,related to dcmsme scheme,"sir, i have not received the otp of pm vishwak...",the tool kit will be distributed through india...


In [16]:
!pip -q install nltk pyspellchecker contractions

In [18]:
# Advanced preprocessing for clustering (English-focused)
# Applies to GrievanceDescription & FinalReply:
# 1) remove stopwords
# 2) spelling correction
# 3) chat/slang treatment via contractions expansion (library)
# 4) URL removal
# 5) remove punctuation
# 6) remove numbers
# 7) handle repeated characters (e.g., 'pleeeease' -> 'pleease')
# 8) remove emails (GrievanceDescription only)
# 9) remove PAN numbers (GrievanceDescription only)


import string
import nltk
from nltk.corpus import stopwords
import contractions
from spellchecker import SpellChecker

# Ensure NLTK stopwords are available
try:
    _ = stopwords.words("english")
except LookupError:
    nltk.download("stopwords")

EN_STOPWORDS = set(stopwords.words("english"))

# Whitelist: domain terms/acronyms we do NOT want spell-corrected
DOMAIN_KEEP = {
    "msme", "udyam", "urn", "nic", "gst", "gstin", "pan", "aadhaar", "aadhar", "upi",
    "llp", "otp", "ifsc", "gov", "india", "portal",
}
# Patterns
EMAIL_RE = re.compile(r"\b[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}\b", re.IGNORECASE)
PAN_RE = re.compile(r"\b[a-z]{5}\s*-?\s*\d{4}\s*-?\s*[a-z]\b", re.IGNORECASE)
URL_RE = re.compile(r"(https?://\S+|www\.\S+)", re.IGNORECASE)
NUM_RE = re.compile(r"\b\d+\b")
REPEATED_RE = re.compile(r"(.)\1{2,}")  # 3+ repeats -> reduce

PUNCT_TABLE = str.maketrans({ch: " " for ch in string.punctuation})

spell = SpellChecker(distance=1)

def _safe_str(x) -> str:
    if x is None or (isinstance(x, float) and pd.isna(x)) or pd.isna(x):
        return ""
    return str(x)

def _only_englishish_token(tok: str) -> bool:
    # Restrict spell correction to simple alphabetic tokens
    return tok.isalpha() and tok.isascii()

@lru_cache(maxsize=200000)
def _correct_token(tok: str) -> str:
    # Keep domain terms as-is
    if tok in DOMAIN_KEEP:
        return tok
    # Spell-correct only for reasonably sized words
    if len(tok) <= 3:
        return tok
    if not _only_englishish_token(tok):
        return tok
    corrected = spell.correction(tok)
    return corrected if corrected else tok

def preprocess_text(text: str, *, remove_emails: bool = True, remove_pan: bool = True) -> str:
    text = _safe_str(text).strip()
    if not text:
        return ""
    # Lowercase (English output expected after translation cell)
    text = text.lower()
    # 8 & 9) remove emails and PAN numbers if present in the GrievanceDescription
    if remove_emails:
        text = EMAIL_RE.sub(" ", text)
    if remove_pan:
        text = PAN_RE.sub(" ", text)
    # 4) URL removal
    text = URL_RE.sub(" ", text)

    # 3) slang/chat treatment using existing library (contractions expansion)
    # Examples: "can't" -> "cannot", "i'm" -> "i am"
    # (This does not expand SMS shortcuts like u->you; add a small map only if needed later.)
    try:
        text = contractions.fix(text)
    except Exception:
        pass

    # 7) handle repeated characters (e.g., 'pleeeease' -> 'pleease')
    # Reduce 3+ repeats to 2 repeats (keeps some emphasis but normalizes)
    text = REPEATED_RE.sub(r"\1\1", text)

    # 6) remove standalone numbers
    text = NUM_RE.sub(" ", text)

    # 5) remove punctuation
    text = text.translate(PUNCT_TABLE)

    # Tokenize (simple whitespace)
    tokens = [t for t in text.split() if t]

    # 2) spelling correction (token-wise, cached)
    corrected = [_correct_token(t) for t in tokens]

    # 1) stopword removal
    filtered = [t for t in corrected if t not in EN_STOPWORDS]

    return " ".join(filtered).strip()

# Defensive checks
missing_cols = [c for c in [DESC_COL, REPLY_COL] if c not in df.columns]
if missing_cols:
    raise KeyError(f"Missing expected columns in df: {missing_cols}. Available columns: {list(df.columns)}")
#df[CATEGORY_COL] = df[CATEGORY_COL].apply(preprocess_text)
df[DESC_COL] = df[DESC_COL].apply(preprocess_text)
df[REPLY_COL] = df[REPLY_COL].apply(preprocess_text)

# Preview
df[[CATEGORY_COL,DESC_COL, REPLY_COL]].head(10)

,Gims_CategoryName,GrievanceDescription,FinalReply
0,uam/udyam registration/certificate related issues,please find attachment letter head,sir please upload clear image aadhar card pan ...
1,uam/udyam registration/certificate related issues,udyam udyam br address wrong want cancel prere...,sir per request advised send details signed co...
2,uam/udyam registration/certificate related issues,udyog aadhaar registration authority dear sir ...,grievance change registered mobile number requ...
3,uam/udyam registration/certificate related issues,regarding udyam registration even multiple att...,provided pan registered udyam udyam cg satan k...
4,uam/udyam registration/certificate related issues,grievance regarding udyam registration even mu...,provided pan registered udyam udyam cg satan k...
5,uam/udyam registration/certificate related issues,dear sir proprietor pan aadhar number trying r...,madam regarding grievance registered champions...
6,uam/udyam registration/certificate related issues,apply udyam registration shows udyam registrat...,sir following information available database d...
7,uam/udyam registration/certificate related issues,sir madam subham aggarwal son fate chandra for...,sir per records uam registered given pan detai...
8,uam/udyam registration/certificate related issues,running cement business want get registered ms...,udyam registration number udyam rj verify udya...
9,related to dcmsme scheme,sir received otp pm vishwakarma yojana take to...,tool kit distributed india post per guideline ...


In [19]:
# Filter the main category (largest number of entries)
dfx = df[df[CATEGORY_COL] == 'uam/udyam registration/certificate related issues']
dfx.shape

(7663, 8)

In [ ]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
# Using the GPU for Clustering
import torch
torch.cuda.is_available(), torch.cuda.get_device_name(0) if torch.cuda.is_available() else None

In [ ]:
# Sub-Categorization of GrievanceDescription which are related to 'UAM Udyam Registration Certificate Related Issues'
from transformers import pipeline

# load zero-shot-classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
UNKNOWN_LABEL = "Unable to identify Semantic meaning"
OTHER_LABEL = "other issues"
# category definitions
candidate_labels = [
    "Email Id, Mobile Number Update related issue",
    "aadhar/PAN Update related issues",
    "PAN already used in the registration"
    "Cancel request for Udyam Certificate Registration",
    "Udyam Registration done Yet not getting Registration ID",
    "Update Udyam registration id",
    "Issues in certificate Downloading",
    "Fake MSME Certificate related Issues",
    "latitude / longitude map Issues",
    "change of address",
    "OTP validation Authentication Error",
    OTHER_LABEL,
]

THRESHOLD = 0.35
dfx = dfx.reset_index(drop=True)
dfx['Sub_Category'] = UNKNOWN_LABEL

for i, desc in dfx[DESC_COL].items():
    if pd.isna(desc) or not str(desc).strip():
        dfx.at[i, 'Sub_Category'] = UNKNOWN_LABEL
        continue
    try:
        output = classifier(str(desc), candidate_labels)
        best_label = output['labels'][0]
        best_score = output['scores'][0]
        dfx.at[i, "Sub_Category"] = best_label if best_score >= THRESHOLD else UNKNOWN_LABEL
    except Exception as e:
        dfx.at[i, 'Sub_Category'] = UNKNOWN_LABEL
    
# show results
dfx[['GrievanceDescription', 'Sub_Category']].head(10)



Device set to use cpu


ValueError: You must include at least one label and at least one sequence.

In [ ]:
df['Gims_CategoryName'].value_counts()

In [ ]:
dfy = df[df[CATEGORY_COL] == 'uam/udyam registration/certificate related issues']